In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
%reload_ext autoreload

In [4]:
import logging
import ArgConfigParse
from pathlib import Path
import os
import fnmatch
import multiprocessing
import time
import csv
import sys
import shutil

In [5]:
import constants

In [6]:
logger = logging.getLogger(__name__)
logger.root.setLevel('DEBUG')

In [7]:
def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = Path(root)/Path(basename)
                yield filename

In [8]:
def doExit(message='unknown error in unknown module: BSoD!', exit_level=0, testing=False):
    logger = logging.getLogger(__name__)
    
    logger.info('exiting before program completion with exit code {}'.format(exit_level))
    print('progam exited due to errors -- see the logs')
    if not testing:
        sys.exit(0)

In [9]:
def tick(n=10):
    for i in range(n):
        print(f'tick: {i} of {n-1}')
        time.sleep(1)

In [10]:
def mapHeaders(file_csv, expected_headers=[]):
    '''map an expected list of header values to their position in a csv
    accepts:
        file_csv (filename) - list containing CSV
        expected_headers (list) - list of headers to search for, ignoring all others
    '''
    logger = logging.getLogger(__name__)
    logger.debug('mapping headers')
    
    
    missing_headers = []
    header_map = {}
        
    try:
        csvHeader = file_csv[0]
    except IndexError as e:
        logger.warning('csv empty: {}'.format(e))
        return(False)
        
    logger.debug('checking for missing headers')
    for each in expected_headers:
        if each not in csvHeader:
            logger.debug('missing: {}'.format(each))
            missing_headers.append(each)

    if len(missing_headers) > 0:
        logging.warning(f'missing expected headers: {missing_headers}')
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            header_map[value] = index
            
    logger.debug('completed mapping headers')
    return(header_map, missing_headers)

In [11]:
def readCSV(csvFile):
    csvFile = Path(csvFile).expanduser()
    file_csv = []
    try:
        with open(csvFile, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                file_csv.append(row)
    except (OSError, IOError) as e:
        logging.error(f'could not read file: {csvFile}')
        logging.error(f'error: {e}')
        return(False)
    
    return(file_csv)

In [12]:
def checkSentry(sentryFile, basePath):
    fileFound = False
    sentryFile = Path(sentryFile)
    basePath = Path(basePath)
        
    for filename in find_files(basePath, '*.txt'):
        if sentryFile.name in filename.name:
            logging.info(f'sentry file ({sentryFile}) found')
            fileFound = filename.parent
            break
    if not fileFound:
        logging.warning(f'sentry file ({sentryFile}) not found in {basePath}')
    return(fileFound)

In [13]:
def createDirectories(studentCSV, headers, studentRoot, subDirs):
    dirCheck = {}
    logging.debug('creating student directories as needed')
    for student in studentCSV[1:]:
        LastFirst = student[headers['LastFirst']]
        ClassOf = 'ClassOf-'+student[headers['ClassOf']]
        Student_Number = student[headers['Student_Number']]
        studentDir = f'{LastFirst} - {Student_Number}'    
        studentDir = studentRoot/ClassOf/studentDir
        dirCheck[studentDir] = {}
        for subDir in subDirs:
            gradeLevelDir = studentDir/subDir
            try:
                gradeLevelDir.mkdir(parents=True)
                action = 'created'
                logging.debug(f'created: {gradeLevelDir}')
            except (FileExistsError):
                action = None
            except Exception as e:
                logging.error(e)
                doExit(e, 1)
                
            
            dirCheck[studentDir][subDir] = action
    return(dirCheck)

In [ ]:
### add this into the main()
appName = constants.appName
develName = constants.develName
userPath = constants.userPath
version = constants.version
configFile = constants.configFile

baseConfig = Path(configFile).resolve()
userConfig = Path('~/.config/').expanduser()/userPath/configFile

# create the user config files if missing
if not userConfig.is_file():
    logging.info(f'creating user config file: {userConfig}')
    try:
        userConfig.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(baseConfig, userConfig)
    except Exception as e:
        doExit(f'failed to create user config file: {e}', 1)
        

configParser = ArgConfigParse.ConfigFile([baseConfig, userConfig])
try:
    configParser.parse_config()
except Exception as e:
    doExit(f'error reading config files: {e}')

config = configParser.config_dict
if not config['main']['google_drive']:
    print('get google drive here')

In [ ]:
def main():
    ## FIXME move this into configuration/interrogation
    basePath = Path('/Volumes/GoogleDrive/Shared drives/IT Blabla I/')
#     basePath = Path('/Volumes/GoogleDrive/Shared drives/ASH Student Cumulative Folders')
    #basePath = Path('/Volumes/GoogleDrive/Shared drives/')
    sentryFile = 'sentryFile_DO_NOT_REMOVE.txt'
    csvFile = './student.export.text'

    subDirs = ['00-Preschool', '00-Transition Kindergarten', '00-zKindergarten', 
               '01-Grade', '02-Grade', '03-Grade', '04-Grade', '05-Grade', '06-Grade', 
               '07-Grade', '08-Grade', '09-Grade', '10-Grade', '11-Grade', '12-Grade']
    
    expectedHeaders = ['ClassOf', 'Student_Number', 'LastFirst']
    
#     config = 

    
    ## FIXME - Add a timeout around this
    studentRoot = checkSentry(sentryFile, basePath)
    if not studentRoot:
        logging.warning(f'This Google Shared Drive ({basePath}) does not appear to include the required sentry file: {sentryFile}')
        logging.warning('Try selecting a different Google Shared Drive.')
        doExit()
    
    ## FIXME handle false returns gracefully
    studentCSV = readCSV(csvFile)
    ## FIXME handle missing headers gracefully
    headers, missing = mapHeaders(studentCSV, expectedHeaders)

    log = createDirectories(studentCSV=studentCSV, headers=headers, studentRoot=studentRoot, subDirs=subDirs)
    
    
    results = {}
    actions = 0
    for student in log:
        created = 0
        for subdir in log[student]:
            if log[student][subdir]:
                created += 1
        results[student] = created
    for each in results:
        if results[each] > 0:
            print(f'created or updated:\n {each}\n  created: {results[each]} folders')
        else:
            print(f'no action needed for:\n {each}')    
    return(True)

In [ ]:
if __name__ == '__main__':
    f = main()


In [ ]:
# # kill a sub process if it's taking too long to complete
# p = multiprocessing.Process(target=tick, name='ticker', args=(3,))
# p.start()

# time.sleep(5)
# if p.is_alive():
#     print('tick is running; killing')
#     p.terminate()
# else:
#     print('tick completed without the need to kill it')
# p.join()